In [41]:
import json
import os
import pandas as pd
import yaml
import re
PROJECT_FOLDER="src"
DEBUG=False

In [42]:
with open(f"../{PROJECT_FOLDER}/pbt_project.yml",'r') as yamlfile:
        cur_yaml = yaml.safe_load(yamlfile)

In [66]:

## Loops through the jobs and pulls out the pipeline information (which job calls which pipeline) with which settings
## from each job's json file
## converts the pipeline technical name to the UI name
## Log the name of the job as well as the name of the individual steps

def extractJobDetails(pipeLineConfig):
    tmpList = [] 
    for key, entry in pipeLineConfig["processes"].items():
        configArray = {}
        configArray["Job UI Name"] = pipeLineConfig["metainfo"]["name"]
        configArray["Step UI Name"]  = entry["metadata"]["slug"]
        try: 
            configArray["Pipeline Folder"] = entry["properties"]["pipelineId"]
            configArray["Pipeline UI Name"] = cur_yaml["pipelines"][entry["properties"]["pipelineId"]]["name"]
        except:
            configArray["Pipeline Folder"] = ""
            configArray["Pipeline UI Name"] = ""
            print("ERROR "+entry["properties"]["pipelineId"])
        #job name
        #print(pipeLineConfig["metainfo"]["name"])
        #process step name
        #print(entry["metadata"]["slug"])
        #process step pipeline
        #print(entry["properties"]["pipelineId"])
        #config settings
        try:
            for key2, config in entry["properties"]["configurations"].items():
                if key2 == "schema":
                    for configValue in config:
                        configArray[configValue["name"]] = configValue["kind"]["value"]
                        #print(configValue["name"])
                        #print(configValue["kind"]["value"])
                        #print("\n")
                if key2 == "overrides":
                    for overKey, overEntry in config.items():
                        configArray[overKey] = overEntry
                        #print(overEntry)
                        #print("\n")
        except:
            pass
        tmpList.append(configArray)
    if DEBUG:    
        print(tmpList)
    return tmpList

def extractPipelinePackageName():
    directory = f"../{PROJECT_FOLDER}/pipelines/"
    tmpList = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isdir(f):
            try:
                #print(f+"/code/setup.py")
                fo = open (f + "/code/setup.py", "r")
                setupFile = fo.read()
                pipelineName = re.findall("name = '(.*)',", setupFile)[0]
                packageName = re.findall("'main = (.*)\.",setupFile)[0]
                #print(re.findall("name = '(.*)',", setupFile))
                #print(re.findall("'main = (.*)\.",setupFile))
                tmpList.append({"packageWheelName": pipelineName ,"prettyName": cur_yaml["pipelines"]["pipelines/"+filename]["name"] , "pipeline":"pipelines/"+filename,"packageName":packageName})
                #pipeLineConfig = json.load(fo)
                #print(pipeLineConfig)
                #configList.extend(extractJobDetails(pipeLineConfig))
            except:
                print("ERROR:" + filename)
                pass
    return tmpList


def extractDatabricksDetails(pipeLineConfig):
    tmpList = [] 
    for item in databrickJobConfig["components"]:
         tmpList.append({"path":item["PipelineComponent"]["path"],"id":item["PipelineComponent"]["id"],"jobPretty": databrickJobConfig["request"]["name"]})
    #print(tmpList)
    return tmpList

def findInListByCriterion(test_list, search, criterion):
    res = None
    
    for sub in test_list:
        if sub[criterion] == search:
            res = sub
            break        
    return res

In [67]:
directory = f"../{PROJECT_FOLDER}/jobs/"
configList = []
databricksJobFiles = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isdir(f):
        try:
            fo = open (f + "/code/prophecy-job.json", "r")
            dbFo = open (f + "/code/databricks-job.json", "r")
            pipeLineConfig = json.load(fo)
            databrickJobConfig = json.load(dbFo)
             
            databricksJobFiles.extend(extractDatabricksDetails(databrickJobConfig))
            configList.extend(extractJobDetails(pipeLineConfig))
        except:
            pass
            print("ERROR: "+f)
        
 ## "-1.0-py3-none-any.whl"


#databricksJobFiles.extend(extractDatabricksDetails(databrickJobConfig))
df = pd.DataFrame(configList)

df3=pd.pivot_table(df.rename(columns={'Step UI Name': 'Entity'}).loc[df['Job UI Name'].str.contains('PIPELINE', case=False, regex=False, na=False)], columns=['sourceSystem'], index=['Entity'], aggfunc='size', fill_value=0).style 
df2 = pd.DataFrame(extractPipelinePackageName())
#df.to_excel("package_details.xlsx")
df4 =  df2[df2.duplicated(['packageName'])]



pipelinePackageName = extractPipelinePackageName()

#f = open("src/jobs/HM2_PIPELINE_JOB/code/prophecy-job.json", "r")
#pipeLineConfig = json.load(f)


ERROR: ../src/jobs/DEV_BBM_PIPELINE_JOB
ERROR: ../src/jobs/DEV_MTK_PIPELINE_JOB
ERROR: ../src/jobs/DEV_ANS_PIPELINE_JOB
ERROR: ../src/jobs/DEV_VST_PIPELINE_JOB
ERROR: ../src/jobs/DEV_LNX_PIPELINE_JOB
ERROR: ../src/jobs/DEV_JNS_PIPELINE_JOB


In [68]:
##Extract the generated databricks job filepaths
##Loop through each file and confirm it is going to exist in DBFS by looking at the current configuration for each pipeline and what it is going to build


##Inputs
## list of pipelines & their respective package details

compareJobWheelAndPipeWheel = []
for item in databricksJobFiles:
    packageDetails = findInListByCriterion(pipelinePackageName,item["id"],"pipeline")
    pipelineGeneratedWheelName = packageDetails["packageWheelName"]
    wheelNameInJob = item["path"].replace("dbfs:/FileStore/prophecy/artifacts/jnj/cp/__PROJECT_ID_PLACEHOLDER__/__PROJECT_RELEASE_VERSION_PLACEHOLDER__/pipeline/","").replace("-1.0-py3-none-any.whl","")
    if pipelineGeneratedWheelName!=wheelNameInJob :
        compareJobWheelAndPipeWheel.append({"Job UI Name": item["jobPretty"], "Pipeline Tech Folder": item["id"], "Pipeline Wheel":packageDetails["packageWheelName"], "Job Wheel": wheelNameInJob, "Mismatched": True})
    else:
        compareJobWheelAndPipeWheel.append({"Job UI Name": item["jobPretty"], "Pipeline Tech Folder": item["id"], "Pipeline Wheel":packageDetails["packageWheelName"], "Job Wheel": wheelNameInJob, "Mismatched": False})
        
    
#databricksJobFiles.extend(extractDatabricksDetails(databrickJobConfig))
df7 = pd.DataFrame(compareJobWheelAndPipeWheel)
df7[df7["Mismatched"]]

,Job UI Name,Pipeline Tech Folder,Pipeline Wheel,Job Wheel,Mismatched


In [69]:
df5 = pd.DataFrame(configList)


#df['C'] = df.apply(lambda x: x.source_system in x.Object, axis=1)
cleaned_column_names = (df5.columns
                        .str.strip()
                        .str.replace('((?<=[a-z0-9])[A-Z]|(?!^)[A-Z](?=[a-z]))', r'_\1')
                        .str.lower()
                        .str.replace('[ _-]+', '_')
                        .str.replace('[}{)(><.!?\\\\:;,-]', ''))

#print(cleaned_column_names)
df5.columns = cleaned_column_names

df5['SourceCheck'] = df5.apply(lambda x: str(x.source_system).upper() in str(x.job_ui_name).upper(), axis=1)
df5['ObjectCheck'] = df5.apply(lambda x: str(x.step_ui_name).upper() in str(x.pipeline_ui_name).upper(), axis=1)
df5 = df5.loc[df5['job_ui_name'].str.contains('PIPELINE', case=False, regex=False, na=False)]
df5 = df5[['SourceCheck', 'ObjectCheck'] + [x for x in df5.columns if x not in ['SourceCheck', 'ObjectCheck']]]
df5 = df5.loc[~(df5['step_ui_name'].isin(['DummyEntry']))]
df5

,SourceCheck,ObjectCheck,job_ui_name,step_ui_name,pipeline_folder,pipeline_ui_name,source_system,mandt,source_database,dai_etl_id,...,exch_rt_diff_amt,qty2,acc_at_orig,valut_goods_rcpt_blok_stk,amt_in_doc_crncy,pcdr_for_updt_sched_line_qty,dbtable7,bom_vld_to_dttm,dbname,tbname
0,True,True,ATL_PIPELINE_JOB,MD_SLS_ORDR_LINE,pipelines/SAP_01_MD_SLS_ORDR_LINE_ATL,SAP_01_MD_SLS_ORDR_LINE_ATL,atl,100,atl,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,ATL_PIPELINE_JOB,MD_SLS_DOC_PTNR_FUNC,pipelines/SAP_01_MD_SLS_DOC_PTNR_FUNC,SAP_01_MD_SLS_DOC_PTNR_FUNC,atl,100,atl,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,ATL_PIPELINE_JOB,MD_DELV_LINE,pipelines/SAP_MD_DELV_LINE_FSN_GEU_HCS_MRS_P01...,SAP_MD_DELV_LINE_FSN_GEU_HCS_MRS_P01_SVS_ATL,atl,100,atl,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,True,ATL_PIPELINE_JOB,MD_PO_SCHED_LINE_DELV,pipelines/SAP_01_MD_PO_SCHED_LINE_DELV,SAP_MD_PO_SCHED_LINE_DELV_ATL_BBA_BBL_BBN_BWI_...,atl,100,atl,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,True,ATL_PIPELINE_JOB,MD_SER_NUM_STOCK_SGMNT,pipelines/SAP_01_MD_SER_NUM_STOCK_SGMNT,SAP_MD_SER_NUM_STOCK_SGMNT_ATL_BBA_BBL_BBN_MRS,atl,100,atl,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,True,True,HCS_PIPELINE_JOB,MD_SLS_ORDR_LINE,pipelines/SAP_01_MD_SLS_ORDR_LINE_hcs,SAP_01_MD_SLS_ORDR_LINE_hcs,hcs,100,hcs,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455,True,True,HCS_PIPELINE_JOB,MD_SER_NUM_STOCK_SGMNT,pipelines/SAP_02_MD_SER_NUM_STOCK_SGMNT,SAP_MD_SER_NUM_STOCK_SGMNT_HCS_MBP_P01_SVS,hcs,100,hcs,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456,True,True,HCS_PIPELINE_JOB,MD_MATL_LOC,pipelines/SAP_MD_MATL_LOC_TAI_MRS_P01_HCS_BWI_...,SAP_MD_MATL_LOC_TAI_MRS_P01_HCS_BWI_GEU_SVS_MB...,hcs,100,hcs,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,True,True,HCS_PIPELINE_JOB,MD_SLS_DOC_PTNR_FUNC,pipelines/SAP_01_MD_SLS_DOC_PTNR_FUNC,SAP_01_MD_SLS_DOC_PTNR_FUNC,hcs,100,hcs,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
directory = f"../{PROJECT_FOLDER}/pipelines/"
pipelineConfigSets = []
systemNames = []




for pipelineFolder in os.listdir(directory):
    f = os.path.join(directory, pipelineFolder)
    # checking if it is a file
    if os.path.isdir(f):
            #print(f+"/code/setup.py")
            with open(f+"/pbt_aspects.yml",'r') as yamlfile:
                aspYAML = yaml.safe_load(yamlfile)
            pipelineConfigList = aspYAML["configurations"]
            pipelineConfigList.remove("default")
            systemNames.extend(pipelineConfigList)

strSearch ="((MES_)?MD_.*?_)("
for systemName in set(systemNames):
    strSearch += systemName +"|"
strSearch =strSearch+"\d{1,2})"
#print(strSearch)
for pipelineFolder in os.listdir(directory):
    f = os.path.join(directory, pipelineFolder)
    # checking if it is a file
    if os.path.isdir(f):

        try:
            #print(f+"/code/setup.py")
            with open(f+"/pbt_aspects.yml",'r') as yamlfile:
                aspYAML = yaml.safe_load(yamlfile)
            pipelineConfigList = aspYAML["configurations"]
            pipelineConfigList.remove("default")
            
            result = re.findall(strSearch,cur_yaml["pipelines"]["pipelines/"+pipelineFolder]["name"])[0][0][:-1]
            #print(cur_yaml["pipelines"]["pipelines/"+pipelineFolder]["name"]+">>>"+str(result))
            potentialObject = pipelineFolder.replace("SAP_","").replace("_01","").replace("_02","").replace("_bwi","")
            for item in pipelineConfigList:
                pipelineConfigSets.append({"Pipeline UI Name":  cur_yaml["pipelines"]["pipelines/"+pipelineFolder]["name"], 
                                           "Pipeline Config Set": item, 
                                           "Pipeline Folder": pipelineFolder,
                                           "Potential Object": result
                                          })
            

                #print("OK------" + pipelineFolder + ":" + str(i))
        except:
             print("ERROR:" + f)
df9 = pd.DataFrame(pipelineConfigSets)

df10 = df9[df9.duplicated(subset=['Potential Object','Pipeline Config Set'], keep=False)]
df10


#print(pipelineConfigSets)
#df8 = pd.DataFrame(pipelinesWithMultiplePackages)
#df8

ERROR:../src/pipelines/SAP_01_MD_BOM_HDR
ERROR:../src/pipelines/SAP_01_MD_SLS_DOC_PTNR_FUNC


,Pipeline UI Name,Pipeline Config Set,Pipeline Folder,Potential Object
155,SAP_MD_MATL_ATL_BWI_FSN_GEU_MRS_P01_SVS_TAI,tai,MD_MATL_SAP_ATL_BWI_FSN_GEU_MRS_P01_SVS_TAI,MD_MATL
170,SAP_MD_MATL_TAI,tai,SAP_MD_MATL_TAI,MD_MATL
190,SAP_MD_BILL_DOC_HDR_BBA_BBN,BBN,SAP_MD_BILL_DOC_HDR_BBA_BBN,MD_BILL_DOC_HDR
191,SAP_MD_BILL_DOC_HDR_BBA_BBN,BBA,SAP_MD_BILL_DOC_HDR_BBA_BBN,MD_BILL_DOC_HDR
287,SAP_MD_MATL_INV_BBA_BBL_BBN_GEU_MRS,MBP,SAP_MD_MATL_INV_BBA_BBL_BBN_GEU_MRS_MBP,MD_MATL_INV
321,SAP_MD_MATL_INV_MBP,MBP,SAP_MD_MATL_INV_MBP,MD_MATL_INV
345,SAP_MD_BILL_DOC_HDR_HMD,BBA,SAP_01_MD_BILL_DOC_HDR,MD_BILL_DOC_HDR
348,SAP_MD_BILL_DOC_HDR_HMD,BBN,SAP_01_MD_BILL_DOC_HDR,MD_BILL_DOC_HDR


In [169]:
##We need to get a pipeline .. figure out what object it is for .. and then pull out all of the configs that are associated with that pipeline
directory = f"../{PROJECT_FOLDER}/pipelines/"
pipelinesWithMultiplePackages = []
for pipelineFolder in os.listdir(directory):
    f = os.path.join(directory, pipelineFolder)
    # checking if it is a file
    if os.path.isdir(f):
        #try:
            i=0
            
            
            for pipeLineFile in os.listdir(f+"/code"):
#                 if pipelineFolder == "SAP_01_MD_SLS_ORDR_LINE_bwi":
#                    print(pipeLineFile)
                if not pipeLineFile == ".DS_Store" and not pipeLineFile == "prophecy" :
                    i+=1 
                
            if i != 7:
                pipelinesWithMultiplePackages.append({"Pipeline Folder":pipelineFolder, "extraPackages":i-7})
                #print(pipelineFolder + ":" + str(i))
            else:
                pass
                #print("OK------" + pipelineFolder + ":" + str(i))
#         except:
#             print(f)
#             pass
df8 = pd.DataFrame(pipelinesWithMultiplePackages)
df8

,Pipeline Folder,extraPackages
0,JDE_01_MD_MATL_BOM,1
1,SAP_MD_MATL_BOM,1


In [171]:
with pd.ExcelWriter("release_summary.xlsx") as writer:
    df3.to_excel(writer, sheet_name="Release Summary", index=True)
    df.to_excel(writer, sheet_name="Job Details", index=False)
    
    df2.to_excel(writer, sheet_name="Pipeline Details", index=False)
    df4.to_excel(writer, sheet_name="Dupe Packages", index=False)
    df5.to_excel(writer, sheet_name="Job Check", index=False)
    df7.to_excel(writer, sheet_name="WheelCheck", index=False)
    df8.to_excel(writer, sheet_name="ExtraPackage", index=False)
    df9.to_excel(writer, sheet_name="PipelineConfigs", index=False)
    df10.to_excel(writer, sheet_name="DupPplnConfig", index=False)